In [1]:
from diffusion_deep_dream_research.model.hooked_model_wrapper import HookedModelWrapper
from diffusers import StableDiffusionPipeline
import torch


Triton not installed, using eager implementation of SAE decoder.


/opt/homebrew/Caskroom/miniconda/base/envs/diffusion-deep-dream-research/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# --- Setup Device ---
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    dtype = torch.float16
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    dtype = torch.float32
else:
    device = torch.device("cpu")
    dtype = torch.float32


In [ ]:
model_path = "../models/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=dtype)
pipe = pipe.to(device)

wrapper = HookedModelWrapper.from_layer(
    pipe=pipe,
    target_layer_name="up_blocks.1.attentions.1"
)


In [ ]:
res=wrapper.forward_with_capture(
    prompts=[
        "a dog in a style of van gogh",
        "a house in a style of picasso",
    ],
    num_images_per_prompt=2
)

activations = res.hook_activations
images = res.images

In [ ]:
print(f"Captured activations at timesteps: {list(activations.keys())}")
for t, act in activations.items():
    print(f"Timestep {t}: activation shape {act.shape}")
    print(f"Timestep {t}: activation min {act.min()}, max {act.max()}")

import matplotlib.pyplot as plt

for i, img in enumerate(images):
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Generated Image {i+1}")
    plt.show()